Лабораторна робота 4

1. Написати функцію пошуку випадкового простого числа з заданого інтервалу або
заданої довжини, використовуючи датчик випадкових чисел та тести перевірки на
простоту. В якості датчика випадкових чисел використовуйте вбудований генератор
псевдовипадкових чисел вашої мови програмування. В якості тесту перевірки на простоту
рекомендовано використовувати тест Міллера-Рабіна із попередніми пробними
діленнями. Тести необхідно реалізовувати власноруч, використання готових реалізацій
тестів не дозволяється.


In [4]:
import csv
import random
from math import gcd

def miller_rabin(p, k=5):
    s, d = 0, p - 1
    while d % 2 == 0:
        s += 1
        d //= 2

    for _ in range(k):
        x = random.randint(2, p - 2)

        if gcd(x, p) > 1:
            return False

        x_d = pow(x, d, p)
        if x_d == 1 or x_d == p - 1:
            continue

        is_composite = True
        for r in range(1, s):
            x_d = pow(x_d, 2, p)
            if x_d == p - 1:
                is_composite = False
                break
            if x_d == 1:
                return False

        if is_composite:
            return False

    return True

def random_prime_in_range(start, end):
    with open("numbers.csv", "a", newline="") as csvfile:
        while True:
            num = random.randint(start, end)
            if miller_rabin(num):
                return num
            else:
                    writer = csv.writer(csvfile)
                    writer.writerow([num])

def random_prime_with_bit_length(bits):
    start = 2**(bits - 1)
    end = 2**bits - 1
    return random_prime_in_range(start, end)

2. За допомогою цієї функції згенерувати дві пари простих чисел  p, q і p1, q1 довжини щонайменше 256 біт. При цьому пари чисел беруться так, щоб pq <= p1q1; p і q – прості числа для побудови ключів абонента А, p1 i q1 – абонента B.

In [5]:
def generate_prime_pairs(bits):
    while True:
        p = random_prime_with_bit_length(bits)
        q = random_prime_with_bit_length(bits)
        p1 = random_prime_with_bit_length(bits)
        q1 = random_prime_with_bit_length(bits)

        if p * q <= p1 * q1:
            return (p, q), (p1, q1)

(pre_p, pre_q), (pre_p1, pre_q1) = generate_prime_pairs(256)
print("Прості числа абонента A: p =", pre_p, ", q =", pre_q)
print("Прості числа абонента B: p1 =", pre_p1, ", q1 =", pre_q1)
print("Перевірка: pq =", pre_p * pre_q, "<= p1q1 =", pre_p1 * pre_q1)
print(pre_p * pre_q <= pre_p1 * pre_q1)

Прості числа абонента A: p = 106562740887006293533753879313712322498428562717557560521390860435756979775533 , q = 74149607855001096120706588791234406114565986659988898493905483417164599232693
Прості числа абонента B: p1 = 81579070940952138200790178334452740161301024008395603877883759575533479961003 , q1 = 101354543085568016288545960019324464512088897148407187297540301695885370884291
Перевірка: pq = 7901585448725608336071709460809208458448196545019628810484790103245785580177188347123688889832300667316164503993417992469324245641387164327932851175100369 <= p1q1 = 8268409460565343223293769722917993719039003950182927796397004447865109657153380614776406200638169961390897417123469868482920589724167170761234754305303873
True


3. Написати функцію генерації ключових пар для RSA. Після генерування функція повинна повертати та/або зберігати секретний ключ (d, p,q) та відкритий ключ  (n,e). За допомогою цієї функції побудувати схеми RSA для абонентів А і B – тобто, створити та зберегти для подальшого використання відкриті ключі (e,n) , (e1, n1) та секретні d і d1.

In [6]:
def mod_inverse(e, phi):
    old_r, r = e, phi
    old_s, s = 1, 0
    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
    if old_r != 1:
        raise ValueError("e не має оберненого за модулем phi")
    return old_s % phi

def generate_rsa_key_pair(p, q):
    n = p * q
    phi = (p - 1) * (q - 1)

    while True:
        e = random.randint(2, phi - 1)
        if gcd(e, phi) == 1:
            break

    d = mod_inverse(e, phi)
    return (e, n), (d, p, q)

public_key_A, private_key_A = generate_rsa_key_pair(pre_p, pre_q)
public_key_B, private_key_B = generate_rsa_key_pair(pre_p1, pre_q1)

print("Відкритий ключ абонента A (e, n):", public_key_A)
print("Секретний ключ абонента A (d, p, q):", private_key_A)
print()
print("Відкритий ключ абонента B (e1, n1):", public_key_B)
print("Секретний ключ абонента B (d1, p1, q1):", private_key_B)


Відкритий ключ абонента A (e, n): (7008971479267891464756759884490843449348909833347140129738764000468972519196137046267496592444657950663078258177608170453589159147164676232459358144058345, 7901585448725608336071709460809208458448196545019628810484790103245785580177188347123688889832300667316164503993417992469324245641387164327932851175100369)
Секретний ключ абонента A (d, p, q): (732299613885779590710919346595996342620437682056554303886396149698212721527995176239763666805067771961241306811221149151480769717926487396050208284780713, 106562740887006293533753879313712322498428562717557560521390860435756979775533, 74149607855001096120706588791234406114565986659988898493905483417164599232693)

Відкритий ключ абонента B (e1, n1): (6288205988327314377712485656788415707382944315131050874823239922341247488682785125080551492699025834265679221657182892184074349739349154278279350950354581, 82684094605653432232937697229179937190390039501829277963970044478651096571533806147764062006381699613908974

4. Написати програму шифрування, розшифрування і створення повідомлення з  цифровим підписом для абонентів А і B. Кожна з операцій (шифрування, розшифрування, створення цифрового підпису, перевірка цифрового підпису) повинна бути реалізована окремою процедурою, на вхід до якої повинні подаватись лише ті ключові дані, які  необхідні для її виконання.
За допомогою датчика випадкових чисел вибрати відкрите повідомлення M і
знайти криптограму для абонентів А и B, перевірити правильність розшифрування.  Скласти для А і B повідомлення з цифровим підписом і перевірити його.

In [7]:
def encrypt(message, public_key):
    return pow(message, public_key[0], public_key[1])

def decrypt(ciphertext, private_key):
    return pow(ciphertext, private_key[0], private_key[1] * private_key[2])

def sign(message, private_key):
    return pow(message, private_key[0], private_key[1] * private_key[2])

def verify(message, signature, public_key):
    return message == pow(signature, public_key[0], public_key[1])


5. За допомогою раніше написаних на попередніх етапах програм організувати роботу протоколу конфіденційного розсилання ключів з підтвердженням справжності по  відкритому каналу за допомогою алгоритму RSA. Протоколи роботи кожного учасника  (відправника та приймаючого) повинні бути реалізовані у вигляді окремих процедур, на  вхід до яких повинні подаватись лише ті ключові дані, які необхідні для виконання.
Перевірити роботу програм для випадково обраного ключа  0 < k < n.

In [13]:
def send_key(message, receiver_public_key, sender_private_key):
    signed_message = sign(message, sender_private_key)
    encrypted_signed_message = encrypt(signed_message, receiver_public_key)
    encrypted_message = encrypt(message, receiver_public_key)
    return encrypted_message, encrypted_signed_message

def receive_key(encrypted_message, encrypted_signed_message, receiver_private_key, sender_public_key):
    decrypted_message = decrypt(encrypted_message, receiver_private_key)
    signed_message = decrypt(encrypted_signed_message, receiver_private_key)
    if verify(decrypted_message, signed_message, sender_public_key):
        print("Перевірка підпису успішна")
        return decrypted_message
    else:
        raise ValueError("Невдала перевірка підпису")

# k = random.randint(2, public_key_A[1] - 1)
# print("Випадкове повідомлення k:", k)


# encrypted_message_to_B, encrypted_signed_message_to_B  = send_key(k, public_key_B, private_key_A)
# print("Зашифроване повідомлення:", encrypted_message_to_B)
# print("Зашифроване підписане повідомлення:", encrypted_signed_message_to_B)
#
# try:
#     received_k = receive_key(encrypted_message_to_B, 3114, private_key_B, public_key_A)
#     print("Отриманий ключ:", received_k)
# except ValueError as e:
#     print(e)
# t = 4878134088213678933867295438683034659964434471896246561594220447222018947750912285520166076778902024365462306432249670130336555500458906847917038696135665
# t_en = encrypt(t, private_key_A)
# print(t_en)
server_key = (65537, 82834032811684679062793375026310729342329054894249212258922932252737952799567)
en_mes = 2714548169814680569877082935664314441244766143576572366836958014574656739932922663702092664047737868963079368107391793279018503315652043349199377607487564
sig_mes = 7658073435847314285157099572702923402668278002001764551742084054351297214048410225738322316690101870514125760097393368326089850167646326798606606374873461
receive_key(en_mes, sig_mes, private_key_A, server_key)



Перевірка підпису успішна


12727984711805633016